Imports

In [1]:
import numpy as np
import os

os.environ["OPENCV_LOG_LEVEL"] = "SILENT"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import cv2

# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

from keras import layers
from Segmentation_classification.utils.pconv_layer import PConv2D
# from keras.src import backend
# backend.update
from keras.applications.efficientnet_v2 import EfficientNetV2B0
from livelossplot import PlotLossesKeras
import pandas as pd
import numpy as np

from dvclive import Live
from dvclive.keras import DVCLiveCallback

2023-11-22 05:18:17.946746: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 05:18:17.946797: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 05:18:17.946820: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Preprocessing

In [2]:
cv2.setNumThreads(cv2.getNumThreads())

def extract(mode='train', dim=256):

    def gen_mask(polygon_coords):
        width, height = dim, dim
        # image = np.zeros((height, width), dtype=np.uint8)

        # Create a polygon mask
        polygon_mask = np.zeros((height, width), dtype=np.int32)
        for val, coord in enumerate(polygon_coords, 1):
            cv2.fillPoly(polygon_mask, [coord], val)
        return polygon_mask

    def preproc_polycoord(coord):  # convert and reshape float coordinates into int
        poly_coord_ = np.array(coord).reshape((-1, 2))  # reshape to (x, y)
        # convert coordinates range in (0-1) to image shape ie 256
        poly_coord_ *= dim
        poly_coord_ = np.round(poly_coord_).astype('int32')
        return poly_coord_

    X, Y1, Y2 = [], [], []
    count = 0
    for file in os.scandir(f'dataset/full_teeth/{mode}/labels'):
        with open(file.path, 'r') as dt:
            text = dt.read()

        if len(np_txt := text.split('\n')) == 2:
            cls_txt = [int(np.fromstring(i, sep=' ')[0]) for i in np_txt]
            np_txt = [np.fromstring(i, sep=' ')[1:] for i in np_txt]
        else:
            continue

        poly_coord = [preproc_polycoord(i) for i in np_txt]
        mask_img = gen_mask(poly_coord)
        # plt.imshow(mask_img , cmap='gray')
        # plt.show()

        img = cv2.imread(file.path.replace(
            'labels', 'images').split('.')[0]+'.jpeg')

        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (dim, dim))
            X.append(img)
            Y1.append(mask_img)
            Y2.append(cls_txt)
            count += 1
        elif count > 100000:
            break
        else:
            continue
    
    Y2 = np.array(Y2)
    return np.array(X).astype(np.int32), np.array(Y1), Y2[:, 0], Y2[:, 1]

dim= 256
X_train, Y_train1, Y_train2, Y_train3 = extract(dim=dim)
# X_test, Y_test1, Y_test2, Y_test3 = extract(mode='test', dim=dim)
X_val, Y_val1, Y_val2, Y_val3 = extract(mode='validate', dim=dim)

Segmentation & Classification

In [ ]:
class SegmentImg(layers.Layer):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        x_img_, y_, segment_region = inputs

        x_img_ = tf.cast(x_img_, dtype=tf.int32)
        y_ = tf.cast(y_, dtype=tf.int32)
        segment_region = tf.cast(segment_region, dtype=tf.int32)

        mask = tf.where(tf.logical_or(tf.equal(y_, 0),
                        tf.equal(y_, segment_region)))
        mask_shape = tf.shape(mask)
        mask_shape = tf.tensor_scatter_nd_update(mask_shape, [[1]], updates =[3])
        update = tf.fill(mask_shape, 0)
        # mask_update = tf.tile(tf.expand_dims(update, axis=-1), [1, 3])
        return tf.cast(tf.tensor_scatter_nd_update(x_img_, mask, updates=update), dtype=tf.float32)

class MyByte(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.base_model = EfficientNetV2B0(include_top=False, input_shape=(
            dim, dim, 3), include_preprocessing=False)
        # Block 1
        self.upsamp_1 = layers.UpSampling2D((2, 2))
        self.conv_1 = layers.Conv2DTranspose(
            1280, (2, 2), padding='same', activation='relu')
        self.conv_2 = layers.Conv2DTranspose(
            1280, (2, 2), padding='same', activation='relu')
        # Block 2
        self.upsamp_2 = layers.UpSampling2D((2, 2))
        self.conv_3 = layers.Conv2DTranspose(
            640, (2, 2), padding='same', activation='relu')
        self.conv_4 = layers.Conv2DTranspose(
            640, (2, 2), padding='same', activation='relu')
        self.conv_5 = layers.Conv2DTranspose(
            640, (2, 2), padding='same', activation='relu')
        # Block 3
        self.upsamp_3 = layers.UpSampling2D((2, 2))
        self.conv_6 = layers.Conv2DTranspose(
            512, (2, 2), padding='same', activation='relu')
        self.conv_7 = layers.Conv2DTranspose(
            512, (2, 2), padding='same', activation='relu')
        self.conv_8 = layers.Conv2DTranspose(
            512, (2, 2), padding='same', activation='relu')
        self.conv_9 = layers.Conv2DTranspose(
            512, (2, 2), padding='same', activation='relu')
        # Block 4
        self.upsamp_4 = layers.UpSampling2D((2, 2))
        self.conv_10 = layers.Conv2DTranspose(
            512, (2, 2), padding='same', activation='relu')
        self.conv_11 = layers.Conv2DTranspose(
            512, (2, 2), padding='same', activation='relu')
        self.conv_12 = layers.Conv2DTranspose(
            512, (2, 2), padding='same', activation='relu')
        self.conv_13 = layers.Conv2DTranspose(
            512, (2, 2), padding='same', activation='relu')
        self.conv_14 = layers.Conv2DTranspose(
            512, (2, 2), padding='same', activation='relu')
        # Block 5
        self.upsamp_5 = layers.UpSampling2D((2, 2))
        self.conv_15 = layers.Conv2DTranspose(
            512, (2, 2), padding='same', activation='relu')
        self.conv_16 = layers.Conv2DTranspose(
            512, (2, 2), padding='same', activation='relu')
        self.conv_17 = layers.Conv2DTranspose(
            512, (2, 2), padding='same', activation='relu')
        self.conv_18 = layers.Conv2DTranspose(
            512, (2, 2), padding='same', activation='relu')
        self.conv_19 = layers.Conv2DTranspose(
            512, (2, 2), padding='same', activation='relu')
        self.conv_20 = layers.Conv2DTranspose(
            512, (2, 2), padding='same', activation='relu')
        self.conv_21 = layers.Conv2DTranspose(
            3, (1, 1), activation='softmax')
        
        # Segmentating input image based on predicted masked image
        self.segment_img = SegmentImg()
        # argmax of segmentation and masking
        self.lambdaa = layers.Lambda(lambda u: tf.argmax(u, axis=3))
        self.masking_layer = layers.Masking(mask_value=0, input_shape=(3,))
        # Classification
        self.multiheadattention1 = layers.MultiHeadAttention(num_heads=2, key_dim=4)
        self.multiheadattention2 = layers.MultiHeadAttention(num_heads=2, key_dim=4)
        self.add = layers.Add() 
        self.layernorm = layers.LayerNormalization()

        # pooling
        self.dropout = layers.Dropout(0.5)
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(256, activation='relu')
        self.dense2 = layers.Dense(256, activation='relu')
        self.denseout1 = layers.Dense(4, activation='softmax')
        self.denseout2 = layers.Dense(4, activation='softmax')

    def call(self, inputs, training=True, mask=None):

        if training:
            x_img, y = inputs
        else:
            x_img, y_mask = inputs

        """x = self.base_model(x_img)
        x = self.upsamp_1(x)
        x = self.conv_1(x)
        x = self.conv_2(x)
        x = self.upsamp_2(x)
        x = self.conv_3(x)
        x = self.conv_4(x)
        x = self.conv_5(x)
        x = self.upsamp_3(x)
        x = self.conv_6(x)
        x = self.conv_7(x)
        x = self.conv_8(x)
        x = self.conv_9(x)
        x = self.upsamp_4(x)
        x = self.conv_10(x)
        x = self.conv_11(x)
        x = self.conv_12(x)
        x = self.conv_13(x)
        x = self.conv_14(x)
        x = self.upsamp_5(x)
        x = self.conv_15(x)
        x = self.conv_16(x)
        x = self.conv_17(x)
        x = self.conv_18(x)
        x = self.conv_19(x)
        x = self.conv_20(x)
        segmentation_output = self.conv_21(x)"""


        if training:
            masked_img1 = self.segment_img([x_img, y, 2])
            masked_img2 = self.segment_img([x_img, y, 1])

            clss1 = self.masking_layer(masked_img1)
            for _ in range(5):
                attn_out1 = self.multiheadattention1(clss1, clss1, use_causal_mask=True)
                clss1 = self.add([clss1, attn_out1])
                clss1 = self.layernorm(clss1)
            clss1 = self.flatten(clss1)
            # clss1 = self.dropout(clss1)
            clss1 = self.dense1(clss1)
            output1 = self.denseout1(clss1)

            clss2 = self.masking_layer(masked_img2)
            for _ in range(5):
                attn_out2 = self.multiheadattention2(clss2, clss2, use_causal_mask=True)
                clss2 = self.add([clss2, attn_out2])
                clss2 = self.layernorm(clss2)
            clss2 = self.flatten(clss2)
            # clss2 = self.dropout(clss2)
            clss2 = self.dense2(clss2)
            output2 = self.denseout2(clss2)

        else:
            # y_mask = self.lambdaa(segmentation_output)
            masked_img1 = self.segment_img([x_img, y_mask, 2])
            masked_img2 = self.segment_img([x_img, y_mask, 1])

            clss1 = self.masking_layer(masked_img1)
            for _ in range(1):
                attn_out1 = self.multiheadattention1(clss1, clss1, use_causal_mask=True)
                clss1 = self.add([clss1, attn_out1])
                clss1 = self.layernorm(clss1)
            clss1 = self.flatten(clss1)
            # clss1 = self.dropout(clss1)
            clss1 = self.dense1(clss1)
            output1 = self.denseout1(clss1)

            clss2 = self.masking_layer(masked_img2)
            for _ in range(1):
                attn_out2 = self.multiheadattention2(clss2, clss2, use_causal_mask=True)
                clss2 = self.add([clss2, attn_out2])
                clss2 = self.layernorm(clss2)
            clss2 = self.flatten(clss2)
            # clss2 = self.dropout(clss2)
            clss2 = self.dense2(clss2)
            output2 = self.denseout2(clss2)

        # return [segmentation_output, output1, output2]
        return [output1, output2]


# Create an instance of your custom model
model = MyByte()
# model.output_names=['seg', 'clss1', 'clss2']
model.output_names=['clss1', 'clss2']

In [13]:
class SegmentImg(layers.Layer):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        x_img_, y_, segment_region = inputs

        x_img_ = tf.cast(x_img_, dtype=tf.int32)
        y_ = tf.cast(y_, dtype=tf.int32)
        segment_region = tf.cast(segment_region, dtype=tf.int32)

        mask = tf.where(tf.logical_or(tf.equal(y_, 0),
                        tf.equal(y_, segment_region)))
        mask_shape = tf.shape(mask)
        mask_shape = tf.tensor_scatter_nd_update(mask_shape, [[1]], updates =[3])
        update = tf.fill(mask_shape, 0)
        # mask_update = tf.tile(tf.expand_dims(update, axis=-1), [1, 3])
        return tf.cast(tf.tensor_scatter_nd_update(x_img_, mask, updates=update), dtype=tf.float32)

input1 = layers.Input((dim, dim, 3))
input2 = layers.Input((dim, dim))

masked_img1 = SegmentImg()([input1, input2, 2])
masked_img2 = SegmentImg()([input1, input2, 1])
clss1 = layers.Masking(mask_value=0, input_shape=(3,))(masked_img1)
for _ in range(5):
    attn_out1 = layers.MultiHeadAttention(num_heads=2, key_dim=4)(clss1, clss1, use_causal_mask=True)
    clss1 = layers.Add()([clss1, attn_out1])
    clss1 = layers.LayerNormalization()(clss1)
clss1 = layers.Flatten()(clss1)
# clss1 = self.dropout(clss1)
clss1 = layers.Dense(256, activation='relu')(clss1)
output1 = layers.Dense(4, activation='softmax', name='clss1')(clss1)

clss2 = layers.Masking(mask_value=0, input_shape=(3,))(masked_img2)
for _ in range(5):
    attn_out2 = layers.MultiHeadAttention(num_heads=2, key_dim=4)(clss2, clss2, use_causal_mask=True)
    clss2 = layers.Add()([clss2, attn_out2])
    clss2 = layers.LayerNormalization()(clss2)
clss2 = layers.Flatten()(clss2)
# clss2 = self.dropout(clss2)
clss2 = layers.Dense(256, activation='relu')(clss2)
output2 = layers.Dense(4, activation='softmax', name='clss2')(clss2)

model = tf.keras.Model(inputs=[input1, input2], outputs=[output1, output2])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 256, 256)]           0         []                            
                                                                                                  
 tf_op_layer_segment_img_4/  [(None, 256, 256)]           0         ['input_6[0][0]']             
 Cast_1 (TensorFlowOpLayer)                                                                       
                                                                                                  
 tf_op_layer_segment_img_5/  [(None, 256, 256)]           0         ['input_6[0][0]']             
 Cast_1 (TensorFlowOpLayer)                                                                       
                                                                                            

Training

In [7]:
model_name = str(dim)+'Attention_model_without_segment'
batch_size = 1
"""model.compile(optimizer=tf.optimizers.RMSprop(learning_rate=0.00001),
              loss={'seg': 'sparse_categorical_crossentropy', 'clss1': 'sparse_categorical_crossentropy',
                    'clss2': 'sparse_categorical_crossentropy'},
              metrics={'seg': 'accuracy', 'clss1': 'accuracy', 'clss2': 'accuracy'})
with Live() as live:
      model.fit([X_train, Y_train1], {'seg': Y_train1, 'clss1': Y_train2, 'clss2': Y_train3}, batch_size=batch_size, epochs=50, verbose=1,
          validation_data=(X_val, [Y_val1, Y_val2, Y_val3]), validation_batch_size=batch_size,
          callbacks=[PlotLossesKeras(), DVCLiveCallback(live=live)])

      model.save(model_name + '.keras')
      live.log_artifact(model_name + '.keras', type=model_name)"""

model.compile(optimizer='adam',
              loss={'clss1': 'sparse_categorical_crossentropy',
                    'clss2': 'sparse_categorical_crossentropy'},
              metrics={'clss1': 'accuracy', 'clss2': 'accuracy'})
with Live() as live:
      model.fit([X_train, Y_train1], {'clss1': Y_train2, 'clss2': Y_train3}, batch_size=batch_size, epochs=50, verbose=1,
                validation_data=([X_val, Y_val1], [Y_val2, Y_val3]),
                callbacks=[PlotLossesKeras(), DVCLiveCallback(live=live)])

      model.save(model_name + '.keras')
      live.log_artifact(model_name + '.keras', type=model_name)
# loss, accuracy = model.evaluate(X_test, [Y_test1, Y_test2, Y_test3])
# error = 197871

Train on 509 samples, validate on 113 samples
Epoch 1/50


	MultiheadAttention.ipynb, 768Pconv_model.keras.dvc, output.png


ResourceExhaustedError: 2 root error(s) found.
  (0) RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[1,2,65536,65536] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node multi_head_attention_10/einsum/Einsum}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[loss_1/Identity/_1097]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

  (1) RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[1,2,65536,65536] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node multi_head_attention_10/einsum/Einsum}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add repo

In [ ]:
model.summary()
hist_df = pd.DataFrame(model.history.history)
hist_df.to_csv(model_name + '.csv', index=False)
# model.save("seg_clss_iter1.keras")

In [12]:

256*256

65536

In [ ]:
# model = tf.keras.models.load_model('segmentation.tf')
img = cv2.imread(
    "fullteeth/test/images/0015w00002NZScQAAX_4dff1fe0-4fc1-11eb-9072-8fb3c6ce79ed-checkinsmileon-1687979590788.jpeg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_res = cv2.resize(img, (256, 256)).reshape((1, 256, 256, 3))
# pred = model.predict(img_res)

In [ ]:
img.dtype

In [ ]:
import matplotlib.pyplot as plt

pred_img = np.argmax(pred, 3).reshape((256, 256))
plt.imshow(img_res.reshape((256, 256, 3)))
plt.show()
plt.imshow(pred_img.astype(np.uint))
plt.show()

In [ ]:
import matplotlib.pyplot as plt
resized_img = np.copy(img_res)
contours, _ = cv2.findContours(pred_img.astype(
    np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
largest_contour = max(contours, key=cv2.contourArea)

# Convert the contour to a list of polygon coordinates
polygon_coords = largest_contour.reshape(-1, 2)
# Optionally, you can round the coordinates to integer values
polygon_coords = np.round(polygon_coords).astype(int)
# Print or use the polygon coordinates
print("Polygon Coordinates:")
cv2.drawContours(resized_img, [polygon_coords], -1,
                 (255, 255, 255), thickness=cv2.FILLED)
plt.imshow(resized_img)
plt.show()

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)